# Evaporation - Groundwater Recharge

In [1]:
# read all necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
from IPython.display import display


### Variables, Acronyms
ETP...Potential Evapotranspiration [mm/d] \
ETR...Real(actual) Evapotranspiration [mm/d] \
Re...extraterrestrial radiation [MJ/(m^2 d)] \
S_B_A...The volumetric soil moisture [%] \
awc...available water capacity [mm] \
K_s...water stress coefficient [-] \
PP...Precipitation [mm] \
Re_mm_day...Extraterrestrial radiation [mm/d]\
xrain... \
p...precipitation

In [2]:
# load data from ETP_data.csv
data = pd.read_csv('ETP_data.csv', index_col='datum', parse_dates=True)
ETP = data.ETP
Re = data.Re


In [3]:
#  calculates K_s dependent on the soil water contet theta_0
## simplified K_s is equal to 1, if S_B_A is bigger or equal to theta_0 and is getting
## smaller with S_B_A until equal to 0 for S_B_A smaleer or equal to 0.
## Between 0 and 1, K_s equals the ratio of S_B_A to theta_0
## K_s represents the relation between ETP and ETR

def linear_reduction_function(S_B_A, available_water_capacity, fraction_of_awc=0.7):
    """
    This function calculates the water stress coefficient K_s based on the volumetric soil moisture.
    Parameters
    ----------
    S_B_A : float
        The volumetric soil moisture in %
    available_water_capacity : float
        The volumetric soil moisture at the field capacity in %
    fraction_of_awc : float
        The fraction of the field capacity at which the plant starts to experience water stress (awc =available_water_capacity)
    Returns
    -------
    K_s : float
        The water stress coefficient
    """ 

    # calculate the "tipping point"
    theta_0 = available_water_capacity * fraction_of_awc

    if S_B_A >= theta_0:
        K_s = 1

    elif S_B_A <= 0:
        K_s = 0

    elif S_B_A > 0 and S_B_A < theta_0:
        K_s = (S_B_A ) / (theta_0 )

    return K_s


In [4]:
def update_moisture_content(soilwater_content, PP, ETR, awc):
    
    # check if water holding capacity is exceeded, and if it is: update the awc and calculate the excess water...
    if soilwater_content + PP - ETR > awc:
        updated_value = awc
        excess_water = soilwater_content + PP - ETR - awc
        
    #...else set the exsess water = 0
    else: 
        updated_value = soilwater_content + PP - ETR
        excess_water = 0

    return updated_value, excess_water


In [5]:
def run_simulation(awc=150, p=0.7, xrain=1):

    # setting initial values
    Re_mm_day = Re / 2.45
    Precip = data['N (mm/d)'] * xrain
    
    gw_flux = np.zeros(len(data))
    soil_moisture = np.zeros(len(data))
    
    initial_soil_moisture = awc
    soil_moisture[0] = initial_soil_moisture

    initial_fraction_of_ETP = 1 # K_s ????
    ETR = np.zeros(len(data))
    
    # then update them
    for i in np.arange(1, len(data)):
        
        etp = ETP[i]
        ETR[0] = initial_fraction_of_ETP * etp
        
        factor = linear_reduction_function(soil_moisture[i-1], awc, p)
        ETR[i] = factor * ETP[i]

        soil_moisture[i], excess_water = update_moisture_content(soil_moisture[i-1], Precip[i], ETR[i], awc)

        gw_flux[i] = excess_water

    return Precip, ETP, ETR, soil_moisture, gw_flux


In [6]:
#plotting the data
def plot_simulation(awc=150, p=0.7, xrain=1):
    
    # run the simulation to create data for Precip, ETP, ETR, soil_moisture and gw_flux, 
    # by handing values for awc, p and xrain
    Precip, ETP, ETR, soil_moisture, gw_flux = run_simulation(awc, p, xrain)

    
    # defining the size and spacing
    fig = plt.figure(figsize=(12, 7))

    width_ratio = 0.4
    spacing_vertical = 0.05
    spacing_horizontal = 0.09

    ## calculating left plot width and right plot width considering the spacing
    left_plot_width = (1 - width_ratio - spacing_horizontal)
    right_plot_width = width_ratio

    ## heights and vertical positions for left plots
    height_left = (1 - 4 * spacing_vertical) / 3
    positions_left = [2/3 + spacing_vertical, 1/3 + spacing_vertical, 0 + spacing_vertical]
    ## creating graphs for subplots on the left
    ax1 = fig.add_axes([0, positions_left[0], left_plot_width, height_left])
    ax2 = fig.add_axes([0, positions_left[1], left_plot_width, height_left])
    ax3 = fig.add_axes([0, positions_left[2], left_plot_width, height_left])

    ## Heights and vertical positions for right plots
    height_right = (1 - 2 * spacing_vertical) / 2
    positions_right = [1/2 + spacing_vertical/2, 0 + spacing_vertical-0.05]
    ## creating graphs for subplots on the right
    ax_right1 = fig.add_axes([1 - right_plot_width, positions_right[0], right_plot_width, height_right])
    ax_right2 = fig.add_axes([1 - right_plot_width, positions_right[1], right_plot_width, height_right])  
    ## place labeling for the right plots on the right
    for ax in [ax_right1, ax_right2]:
        ax.yaxis.tick_right()
        ax.yaxis.set_label_position("right")

    
    # Now plot the data
    ## plot plot precipitation
    ax1.bar(data.index, Precip.values, color='skyblue', label='precipitation', width=1)
    ax1.set_ylim(max(Precip), 0)  # This inverts the y-axis

    ## plot the ETP
    ax2.plot(data.index, ETP, color='goldenrod', label='ETP')

    ## plot the ETR
    ax2.plot(data.index, ETR, color='tomato', label='ETR')

    ## plot the soil moisture storage
    ax3.plot(data.index, soil_moisture, color='steelblue', label='soil moisture storage')

    ax4 = ax3.twinx()
    ## plot the groundwater flux
    ax4.bar(data.index, gw_flux, color='navy', label='groundwater flux', width=1)
    S_B_A = np.arange(0, 150, 1)
    curve = np.zeros(len(S_B_A))

    
    # adding cumulative distribution of precipitation and gw_flux
    ## creating cumulativ data
    cumulative_precip = np.cumsum(Precip.values)
    cumulative_gw_flux = np.cumsum(gw_flux)
    ## plotting on the top right 
    ax_right1.plot(data.index, cumulative_precip, color='skyblue', label='cumulative precipitation')
    ax_right1.plot(data.index, cumulative_gw_flux, color='navy', label='cumulative groundwater flux')

    # adding cumulative distribution of ETP and ETR
    ## creating cumulativ data
    cumulative_ETP = np.cumsum(ETP)
    cumulative_ETR = np.cumsum(ETR)
    ## plotting on the bottom right
    ax_right2.plot(data.index, cumulative_ETP, color='goldenrod', label='cumulative ETP')
    ax_right2.plot(data.index, cumulative_ETR, color='tomato', label='cumulative ETR')
 
   
    # add labels and legend
    ##left
    ax1.set_ylabel('precipitation (mm/d)')
    ax2.set_ylabel('ETP and ETR (mm/d)')
    ax3.set_ylabel('soil moisture (mm)')
    ax4.set_ylabel('groundwater flux (mm/d)')
    ax1.legend(loc='best')
    ax2.legend(loc='best')
    ax3.legend(loc='upper left')
    ax4.legend(loc='center right')
    ##right
    ax_right1.set_ylabel('precipitation and gw_flux (mm/d)')
    ax_right2.set_ylabel('ETP and ETR (mm/d)') 
    ax_right1.legend(loc='upper left')
    ax_right2.legend(loc='best') 
    ## labling intervall x-axes right - V
    #ax_right1.xaxis.set_major_locator(ticker.MultipleLocator(160))
    #ax_right2.xaxis.set_major_locator(ticker.MultipleLocator(160))


In [7]:
def on_button_click(button):
    awc_val = awc_slider.value
    p_val = p_slider.value
    xrain_val = xrain_slider.value

    clear_output(wait=True)
    plot_simulation(awc_val, p_val, xrain_val)
    display(interactive_plot)
    display(button)


def create_interactive_plot():
    global awc_slider, p_slider, xrain_slider, interactive_plot

    # Create the button
    button = widgets.Button(description="Update Simulation")
    button.on_click(on_button_click)

    # Create sliders for the parameters
    awc_slider = widgets.FloatSlider(min=0, max=500, step=5, value=150, description='awc:')
    p_slider = widgets.FloatSlider(min=0.5, max=1.0, step=0.1, value=0.7, description='p:')
    xrain_slider = widgets.FloatSlider(min=0.1, max=5, step=0.5, value=1, description='xrain:')

    # Display the interactive plot with the sliders and button
    interactive_plot = interact(plot_simulation, awc=awc_slider, p=p_slider, xrain=xrain_slider)
    display(interactive_plot)

    

In [8]:
create_interactive_plot()


interactive(children=(FloatSlider(value=150.0, description='awc:', max=500.0, step=5.0), FloatSlider(value=0.7…

<function __main__.plot_simulation(awc=150, p=0.7, xrain=1)>